<a href="https://colab.research.google.com/github/MSaber7/DL/blob/master/CCreateModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library

In [0]:
import numpy as np
import os
from keras import regularizers
from keras.datasets import mnist
from keras.layers import *
from keras.models import *
from time import time
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score
from keras import metrics
import tensorflow as tf


from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import (Input, Dense, Concatenate)
from keras.utils import np_utils


Using TensorFlow backend.


### Load / Split Data

In [0]:
nb_classes = 10
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

### Create Model 

In [0]:
def CreateModel(layerNo = 2 , neurons=20 , optimizer='rmsprop', init='glorot_uniform' , activation = 'relu' , lossFun = 'categorical_crossentropy',dropout_rate=0.0 , l1=0.0 , l2=0.0 , learn_rate=0.01, momentum=0):
  
    model = Sequential()
    
    # Input
    model.add(Dense(neurons, activation=activation, input_shape=(784,)))
    model.add(Dropout(0.2))
    
    for i in range (layerNo):
      model.add(Dense(neurons, activation=activation,
                kernel_regularizer = regularizers.l2(l2) ,
                activity_regularizer=regularizers.l1(l2)))
      
      model.add(Dropout(dropout_rate))
    
    # Output
    model.add(Dense(10,init=init)) # init: Weight Initialization
    model.add(Activation('softmax'))
    
    #print(model.summary())
    
    # Compile
    #model.compile(loss=lossFun, optimizer=optimizer, metrics=['accuracy']) 
    
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss=lossFun, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [0]:
CreateModel(layerNo = 2 , neurons=20 , optimizer='rmsprop', init='glorot_uniform' , activation = 'relu' , lossFun = 'categorical_crossentropy',dropout_rate=0.0 , l1=0.0 , l2=0.0 , learn_rate=0.01, momentum=0).summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                15700     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                420       
_________________________________________________________________
dropout_3 (Dropout)  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="glorot_uniform")`


### **GridSearchCV**

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# ----------------------------- Wrappers ---------------------------------------

start=time()
model = KerasClassifier(build_fn=CreateModel)

# ----------------------------- Parameters -------------------------------------

optimizer =  ['SGD', 'RMSprop','Adam', 'Adamax']
activation = ['relu', 'tanh', 'sigmoid', 'linear']
init = ['uniform', 'normal', 'zero', 'he_normal', 'he_uniform']
lossFun = ['categorical_crossentropy','hinge']

neurons = [10, 15]
#epochs = np.array([1, 2])
batches = np.array([5, 10])
dropout_rate = [0.1,0.2]
l1 = [0.0001 , 0.00001]
l2 = [0.0001 , 0.00001]
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

param_grid = dict(optimizer=optimizer, activation = activation, lossFun=lossFun , init=init , 
                  neurons = neurons ,
                  dropout_rate = dropout_rate , batch_size=batches , l1 = l1 , l2=l2  )

# ------------------------------ GridSearchCV ----------------------------------

grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=1)
grid_result = grid.fit(X_train, Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# ----------------------------- Accuracy Score ---------------------------------

y_pred = grid_result.predict(X_test)
print("Accuracy on the test set: {:.2f}%".format(accuracy_score(Y_test, y_pred) * 100))

# -------------------------------- Results -------------------------------------

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="uniform")`


Epoch 1/1
20000/20000 [==============================] - 2s 104us/step
Epoch 1/1
20000/20000 [==============================] - 2s 106us/step
Epoch 1/1
20000/20000 [==============================] - 2s 108us/step
Epoch 1/1
20000/20000 [==============================] - 2s 117us/step
Epoch 1/1
20000/20000 [==============================] - 2s 116us/step
Epoch 1/1
20000/20000 [==============================] - 2s 117us/step
Epoch 1/1
20000/20000 [==============================] - 2s 122us/step
Epoch 1/1
20000/20000 [==============================] - 2s 125us/step
Epoch 1/1
20000/20000 [==============================] - 3s 150us/step
Epoch 1/1
20000/20000 [==============================] - 3s 147us/step
Epoch 1/1
20000/20000 [==============================] - 3s 154us/step
Epoch 1/1
20000/20000 [==============================] - 3s 144us/step
Epoch 1/1
20000/20000 [==============================] - 3s 145us/step
Epoch 1/1
20000/20000 [==============================] - 3s 152us/step
Epoch 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="normal")`


Epoch 1/1
20000/20000 [==============================] - 22s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 22s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 22s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 22s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 22s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 23s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 23s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 23s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 23s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 23s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 23s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 23s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 23s 1ms/step
Epoch 1/1
20000/20000 [==============================] - 24s 1ms/step
Epoch 1/1
20000/2000

In [0]:
y_pred = grid_result.predict(X_test)
print("Accuracy on the test set: {:.2f}%".format(accuracy_score(Y_test, y_pred) * 100))

### **RandomizedSearchCV**

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

# ----------------------------- Wrappers ---------------------------------------
start= time()
model = KerasClassifier(build_fn=CreateModel,verbose=1)

# ----------------------------- Parameters -------------------------------------

n_iter_search = 16 # Number of parameter settings that are sampled.
optimizer =  ['SGD','Adam']
activation = ['relu', 'sigmoid']
#init = ['uniform', 'normal']
#lossFun = ['categorical_crossentropy','hinge']

neurons = [10, 15]
#epochs = np.array([50, 100, 150])
#batches = np.array([5, 10, 20])
#dropout_rate = [0.0 ,0.1,0.2]
#l1 = [0.001 , 0.0001]
#l2 = [0.001 , 0.0001]
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

param_grid = dict(optimizer=optimizer, activation = activation ,neurons = neurons )

# ------------------------------ RandomizedSearchCV ----------------------------

#param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init)

random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param_grid,
                                   n_iter=n_iter_search)

random_search.fit(X_train, Y_train)

print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))

# ---------------------------------- Results -----------------------------------

means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']

print("total time:",time()-start)

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 8 is smaller than n_iter=16. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="glorot_uniform")`


Epoch 1/1
20000/20000 [==============================] - 1s 64us/step
Epoch 1/1
20000/20000 [==============================] - 1s 71us/step
Epoch 1/1
20000/20000 [==============================] - 1s 73us/step
Epoch 1/1
20000/20000 [==============================] - 1s 69us/step
Epoch 1/1
20000/20000 [==============================] - 1s 71us/step
Epoch 1/1
20000/20000 [==============================] - 1s 72us/step
Epoch 1/1
20000/20000 [==============================] - 2s 76us/step
Epoch 1/1
20000/20000 [==============================] - 2s 76us/step
Epoch 1/1
20000/20000 [==============================] - 2s 78us/step
Epoch 1/1
20000/20000 [==============================] - 2s 79us/step
Epoch 1/1
20000/20000 [==============================] - 2s 83us/step
Epoch 1/1
20000/20000 [==============================] - 2s 83us/step
Epoch 1/1
20000/20000 [==============================] - 2s 83us/step
Epoch 1/1
20000/20000 [==============================] - 2s 86us/step
Epoch 1/1
20000/2000

### Create Model with Noise

- Load, reshape, scale and add noise to data,
- Train DAE on merged training and testing data,
- Get neuron outputs from DAE as new features,
- Train classification algorithm on new features.

**Load , scale and reshape of data**

In [0]:
# Load data
from keras.datasets import mnist

# input image dimensions
img_rows, img_cols = 28, 28                          
input_shape = (img_rows * img_cols, )

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


**Add Noise**

In [0]:
noise_factor = 0.4
x_train_noisy = x_train + noise_factor * np.random.normal(size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
x_test_noisy = np.clip(x_test_noisy, 0.0, 1.0)

**CreateModel by Denoise Autoencoder**

In [0]:
def CreateModelDNA ( layerNo = 2 , input_size = 784 , hidden_size = 128 , code_size = 32 , actFun = 'relu' , opts = 'adam' , lossFun = 'binary_crossentropy'):
    
    f = ['o']*layerNo
    b = ['o']*layerNo
    
    Encoder = []
    n = ['Encoder_']*layerNo
    for i in range (layerNo):
      s = n[i] + str(i)
      Encoder.append(s)
      
    Decoder = []
    n = ['Decoder_']*layerNo
    for i in range (layerNo):
      s = n[i] + str(i)
      Decoder.append(s)
    
# ------------------------------- Input ----------------------------------------

    # Input
    input_img = Input(shape=(input_size,),name = 'inputsoo')

# ------------------------------ Encoder ---------------------------------------    

    for i in range (layerNo):
      if i == 0 :
        f[i] = Dense(hidden_size, activation=actFun , name = Encoder[i])(input_img)
      else:
         f[i] = Dense(hidden_size, activation=actFun , name = Encoder[i])(f[i-1])

#------------------------------- Center ----------------------------------------

    code = Dense(code_size, activation=actFun , name = 'Center')(f[i])
  
# ------------------------------ Decoder ---------------------------------------
    
    for i in range (layerNo):
      if i == 0 :
        b[i] = Dense(hidden_size, activation=actFun , name= Decoder[i])(code)
      else:
         b[i] = Dense(hidden_size, activation=actFun , name = Decoder[i])(b[i-1])

# ------------------------------ Output ----------------------------------------

    output_img = Dense(input_size, activation='sigmoid',name = 'outputs')(b[i])
  
# ------------------------------------------------------------------------------

    # Print network summary
    Model(input_img, output_img).summary()

# ---------------------------- Compile -----------------------------------------

    autoencoder = Model(input_img, output_img)
    autoencoder.compile(optimizer=opts, loss=lossFun, metrics = ['mae', 'categorical_accuracy'])
    autoencoder.fit(x_train_noisy, x_train, epochs=10)
    
    return (autoencoder)


In [0]:
 CreateModelDNA ( layerNo = 1 , input_size = 784 , hidden_size = 128 , code_size = 32 , actFun = 'relu' , opts = 'adam' , lossFun = 'binary_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputsoo (InputLayer)        (None, 784)               0         
_________________________________________________________________
Encoder_0 (Dense)            (None, 128)               100480    
_________________________________________________________________
Center (Dense)               (None, 32)                4128      
_________________________________________________________________
Decoder_0 (Dense)            (None, 128)               4224      
_________________________________________________________________
outputs (Dense)              (None, 784)               101136    
Total params: 209,968
Trainable params: 209,968
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
60000